# LLM Tasks
Classify Genre given title

Predict Similar titles given title

Predict Genre Combinations that are not yet popular


In [13]:
from transformers import BertForSequenceClassification, BertTokenizer
from torch_geometric.data import download_url, extract_zip
import torch
import pandas as pd

In [6]:
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, './data'), './data')

movies_path = './data/ml-latest-small/movies.csv'
ratings_path = './data/ml-latest-small/ratings.csv'

Using existing file ml-latest-small.zip
Extracting ./data\ml-latest-small.zip


In [14]:
print('movies.csv:')
print('===========')
movies_df = pd.read_csv(movies_path)
genres = movies_df['genres'].str.get_dummies('|')
print(movies_df.head())

movies.csv:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [12]:
print('ratings.csv:')
print('===========')
ratings_df = pd.read_csv(ratings_path)
print(ratings_df.head())


ratings.csv:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [29]:
# Create a mapping from unique user indices to range [0, num_user_nodes):
unique_user_id = ratings_df['userId'].unique()
unique_user_id = pd.DataFrame(data={
    'userId': unique_user_id,
    'mappedUserId': pd.RangeIndex(len(unique_user_id)),
})
print("Mapping of user IDs to consecutive values:")
print("==========================================")
print(unique_user_id.head())
print()
# Create a mapping from unique movie indices to range [0, num_movie_nodes):
unique_movie_id = pd.DataFrame(data={
    'movieId': movies_df.index,
    'mappedMovieId': pd.RangeIndex(len(movies_df)),
    'title': movies_df["title"]
})
print("Mapping of movie IDs to consecutive values:")
print("===========================================")
print(unique_movie_id.head())

Mapping of user IDs to consecutive values:
   userId  mappedUserId
0       1             0
1       2             1
2       3             2
3       4             3
4       5             4

Mapping of movie IDs to consecutive values:
   movieId  mappedMovieId                               title
0        0              0                    Toy Story (1995)
1        1              1                      Jumanji (1995)
2        2              2             Grumpier Old Men (1995)
3        3              3            Waiting to Exhale (1995)
4        4              4  Father of the Bride Part II (1995)


In [31]:
ratings_df.merge(unique_user_id, on="userId", how="left").merge(unique_movie_id, on="movieId")

,userId,movieId,rating,timestamp,mappedUserId,mappedMovieId,title
0,1,1,4.0,964982703,0,1,Jumanji (1995)
1,1,3,4.0,964981247,0,3,Waiting to Exhale (1995)
2,1,6,4.0,964982224,0,6,Sabrina (1995)
3,1,47,5.0,964983815,0,47,Mighty Aphrodite (1995)
4,1,50,5.0,964982931,0,50,Georgia (1995)
...,...,...,...,...,...,...,...
77934,610,8950,4.5,1493849533,609,8950,Creep (2014)
77935,610,8957,3.0,1493847395,609,8957,Bad Asses on the Bayou (2015)
77936,610,8961,5.0,1493844829,609,8961,Villain (1971)
77937,610,8983,3.5,1493845284,609,8983,Feast (2014)
